In [1]:
import requests
import string
import re

from bs4 import BeautifulSoup #untuk memisahkan teks yang ingin diambil dengan struktur html
import nltk
from nltk.corpus import stopwords

%pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

%pip install plotly
%pip install --upgrade gensim
from gensim.models import Word2Vec, FastText

import numpy as np

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [2]:
web = requests.get("https://kompas.com")
soup = BeautifulSoup(web.content, 'html.parser')
for a in soup(['script', 'style']):
    a.decompose()
text = ' '.join(soup.stripped_strings)
print(text)

Berita Terkini Hari Ini, Kabar Akurat Terpercaya - Kompas.com Otomatis Mode Gelap Mode Terang Login Gabung KOMPAS.com+ Konten yang disimpan Konten yang disukai Atur Minat Berikan Masukanmu Langganan Kompas One News Nasional Global Megapolitan Regional Pemilu Hype Konsultasi Hukum Cek Fakta Surat Pembaca Indeks Kilas Daerah Kilas Korporasi Kilas Kementerian Sorot Politik Kilas Badan Negara Kelana Indonesia Kalbe Health Corner Kilas Parlemen Kilas BUMN Nusaraya Sumatera Utara Sumatera Selatan Sumatera Barat Riau Lampung Banten Yogyakarta Jawa Barat Jawa Tengah Jawa Timur Kalimantan Barat Kalimantan Timur Sulawesi Selatan Bali Indeks Jagat Literasi Artikel Foto Video Rolls Donasi STEM Cahaya Aktual Doa dan Niat Jadwal Sholat Tekno Apps & OS Gadget Internet Hardware Business Game Galeri Indeks Tech Innovation Kilas Internet Otomotif Motor Mobil Sport Niaga Komunitas Otopedia Merapah EV Leadership Elektrifikasi Pameran Bola Timnas Indonesia Liga Indonesia Liga Inggris Liga Italia Liga Champ

In [3]:
teks = text.lower()
teks = re.sub(r'\d+', '', teks)
teks = teks.translate(str.maketrans('', '', string.punctuation))
teks = re.sub(r'\s+', ' ', teks)
teks = teks.strip()

In [4]:
# stemming/lemmatization (mengembalikan kata ke bentuk dasar) untuk Bahasa Indonesia

# Stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stemmed = stemmer.stem(teks)

# Lemmatization (menggunakan stemming hasil Sastrawi, karena lemmatizer khusus Indonesia belum tersedia)
output = stemmed
factory = StemmerFactory()
stemmer = factory.create_stemmer()
output = stemmer.stem(teks)

print("Hasil stemming/lemmatization (Bahasa Indonesia):", output)


KeyboardInterrupt: 

In [16]:
#tokenizing
tokens = [t for t in output.split()]
print(tokens)

['berita', 'kini', 'hari', 'ini', 'kabar', 'akurat', 'percaya', 'kompascom', 'otomatis', 'mode', 'gelap', 'mode', 'terang', 'login', 'gabung', 'kompascom', 'konten', 'yang', 'simpan', 'konten', 'yang', 'suka', 'atur', 'minat', 'ikan', 'masuk', 'langgan', 'kompas', 'one', 'news', 'nasional', 'global', 'megapolitan', 'regional', 'milu', 'hype', 'konsultasi', 'hukum', 'cek', 'fakta', 'surat', 'baca', 'indeks', 'kilas', 'daerah', 'kilas', 'korporasi', 'kilas', 'menteri', 'sorot', 'politik', 'kilas', 'badan', 'negara', 'kelana', 'indonesia', 'kalbe', 'health', 'corner', 'kilas', 'parlemen', 'kilas', 'bumn', 'nusaraya', 'sumatera', 'utara', 'sumatera', 'selatan', 'sumatera', 'barat', 'riau', 'lampung', 'banten', 'yogyakarta', 'jawa', 'barat', 'jawa', 'tengah', 'jawa', 'timur', 'kalimantan', 'barat', 'kalimantan', 'timur', 'sulawesi', 'selatan', 'bal', 'indeks', 'jagat', 'literasi', 'artikel', 'foto', 'video', 'rolls', 'donasi', 'stem', 'cahaya', 'aktual', 'doa', 'dan', 'niat', 'jadwal', 'sho

In [17]:
# menghapus stopwords dari tokens
nltk.download('stopwords')
stop_words = set(stopwords.words('indonesian'))
filtered_tokens = [word for word in tokens if word not in stop_words]
print(filtered_tokens)

['berita', 'kabar', 'akurat', 'percaya', 'kompascom', 'otomatis', 'mode', 'gelap', 'mode', 'terang', 'login', 'gabung', 'kompascom', 'konten', 'simpan', 'konten', 'suka', 'atur', 'minat', 'ikan', 'masuk', 'langgan', 'kompas', 'one', 'news', 'nasional', 'global', 'megapolitan', 'regional', 'milu', 'hype', 'konsultasi', 'hukum', 'cek', 'fakta', 'surat', 'baca', 'indeks', 'kilas', 'daerah', 'kilas', 'korporasi', 'kilas', 'menteri', 'sorot', 'politik', 'kilas', 'badan', 'negara', 'kelana', 'indonesia', 'kalbe', 'health', 'corner', 'kilas', 'parlemen', 'kilas', 'bumn', 'nusaraya', 'sumatera', 'utara', 'sumatera', 'selatan', 'sumatera', 'barat', 'riau', 'lampung', 'banten', 'yogyakarta', 'jawa', 'barat', 'jawa', 'jawa', 'timur', 'kalimantan', 'barat', 'kalimantan', 'timur', 'sulawesi', 'selatan', 'bal', 'indeks', 'jagat', 'literasi', 'artikel', 'foto', 'video', 'rolls', 'donasi', 'stem', 'cahaya', 'aktual', 'doa', 'niat', 'jadwal', 'sholat', 'tekno', 'apps', 'os', 'gadget', 'internet', 'hard

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### tf-idf
TF-IDF (Term Frequency-Inverse Document Frequency): TF-IDF menghitung seberapa penting sebuah kata dalam sebuah dokumen relatif terhadap seluruh koleksi dokumen. Outputnya adalah matriks di mana setiap baris mewakili dokumen dan setiap kolom mewakili kata unik dalam seluruh koleksi. Nilai di dalam sel menunjukkan skor TF-IDF kata tersebut dalam dokumen. Karena setiap kolom mewakili sebuah kata, maka header kolomnya adalah kata-kata itu sendiri.

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(filtered_tokens)

# ubah ke DataFrame biar keliatan
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=vectorizer.get_feature_names_out()
)

print("\nTF-IDF shape:", tfidf_df.shape)
print(tfidf_df.head())

# Simpan hasil TF-IDF ke CSV
# Jika Anda ingin menyimpan dokumen x fitur, indeks baris mewakili dokumen yang diberikan ke fit_transform
tfidf_df.to_csv('tfidf.csv', index=False, float_format='%.6f', encoding='utf-8')


TF-IDF shape: (1456, 796)
   abdullah  about  absen  adik  ads  adu  advertise  agama  agri  agustus  \
0       0.0    0.0    0.0   0.0  0.0  0.0        0.0    0.0   0.0      0.0   
1       0.0    0.0    0.0   0.0  0.0  0.0        0.0    0.0   0.0      0.0   
2       0.0    0.0    0.0   0.0  0.0  0.0        0.0    0.0   0.0      0.0   
3       0.0    0.0    0.0   0.0  0.0  0.0        0.0    0.0   0.0      0.0   
4       0.0    0.0    0.0   0.0  0.0  0.0        0.0    0.0   0.0      0.0   

   ...  wilayah  wirausahawan  wisata  wujud  wuk  wur  xiaomi  yogyakarta  \
0  ...      0.0           0.0     0.0    0.0  0.0  0.0     0.0         0.0   
1  ...      0.0           0.0     0.0    0.0  0.0  0.0     0.0         0.0   
2  ...      0.0           0.0     0.0    0.0  0.0  0.0     0.0         0.0   
3  ...      0.0           0.0     0.0    0.0  0.0  0.0     0.0         0.0   
4  ...      0.0           0.0     0.0    0.0  0.0  0.0     0.0         0.0   

   youtuber  zoni  
0       0.0   0

## word embedding

Word2Vec: Word2Vec adalah teknik word embedding yang memetakan setiap kata ke dalam sebuah vektor numerik dalam ruang berdimensi rendah (dalam kasus Anda, 56 dimensi). Outputnya adalah sekumpulan vektor, satu untuk setiap kata. Tabel yang Anda lihat menampilkan vektor-vektor ini. Setiap baris adalah kata, dan setiap kolom adalah salah satu dari 56 dimensi vektor tersebut. Oleh karena itu, header kolomnya adalah indeks dimensi (0 hingga 55), bukan kata-kata itu sendiri.
Singkatnya, TF-IDF menggunakan kata sebagai fitur, sedangkan Word2Vec menggunakan dimensi vektor sebagai fitur untuk merepresentasikan kata.

In [19]:
corpus = []
for col in filtered_tokens:
   word_list = col.split(" ")
   corpus.append(word_list)

#show first value
corpus[0:1]

#generate vectors from corpus
model = Word2Vec(corpus, min_count=1, vector_size = 56)
print(model)

Word2Vec<vocab=800, vector_size=56, alpha=0.025>


In [20]:
class MyTokenizer:
    def fit_transform(self, texts):
        # Tokenisasi sederhana: lowercase + split
        return [str(text).lower().split() for text in texts]

class MeanEmbeddingVectorizer:
    def __init__(self, word2vec_model):
        self.word2vec = word2vec_model
        # Perbaikan: gunakan vector_size (Gensim ≥ 4.0)
        self.dim = word2vec_model.wv.vector_size

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tokenized = MyTokenizer().fit_transform(X)
        embeddings = []
        for words in X_tokenized:
            # Ambil vektor hanya untuk kata yang ada di vocab
            valid_vectors = [
                self.word2vec.wv[word] for word in words
                if word in self.word2vec.wv
            ]
            if valid_vectors:
                embeddings.append(np.mean(valid_vectors, axis=0))
            else:
                embeddings.append(np.zeros(self.dim))
        return np.array(embeddings)

    def fit_transform(self, X, y=None):
        return self.transform(X)

In [21]:
mean_embedding_vectorizer = MeanEmbeddingVectorizer(model)
mean_embedded = mean_embedding_vectorizer.fit_transform([' '.join(filtered_tokens)])
print(mean_embedded)

[[-2.7003392e-04  4.2531066e-04  4.4232607e-04  4.2322406e-04
   7.7212469e-05 -7.8060210e-04  4.6681592e-04 -8.3155493e-04
  -1.0894785e-03 -8.0118043e-04 -7.6753751e-04 -9.3880153e-05
  -4.6876841e-05 -4.1861497e-04  2.8906083e-05  1.2218597e-03
  -2.7702877e-04  2.4325485e-04  4.1746885e-06 -4.0944316e-05
   5.2286213e-04  4.8105698e-04  5.1001966e-04 -7.3930481e-04
   7.2097656e-04 -3.5313162e-04  1.2111743e-03  7.7221193e-04
  -1.6289912e-04 -5.9976231e-04 -5.7370332e-04  2.9825687e-04
   1.3926017e-04 -5.5644003e-05 -7.7885552e-04 -5.6212716e-04
   7.7202480e-04 -2.4221546e-04  7.2814245e-04 -4.2646125e-04
  -6.7524007e-04 -1.8738526e-04  3.0446419e-05  4.7901477e-04
  -1.8967327e-04  4.5508673e-04  5.8436219e-04  7.4628787e-04
   2.8179557e-04  5.2998320e-04 -5.3852674e-04  7.1011134e-04
  -2.6477117e-04  9.3438757e-05  2.0440969e-04 -4.8174334e-04]]


In [22]:
# Ambil semua fitur (kata unik) dari model
features = list(model.wv.index_to_key)

# Ambil vektor embedding untuk setiap fitur
embedding_vectors = [model.wv[word] for word in features]

# Buat DataFrame, setiap kolom adalah dimensi vektor
embedding_df = pd.DataFrame(embedding_vectors, index=features)
embedding_df.index.name = 'fitur'
print(embedding_df)

# Simpan per-term embeddings ke CSV
embedding_df.reset_index().to_csv('cbow_terms.csv', index=False, float_format='%.6f', encoding='utf-8')
print('Saved cbow_terms.csv')

                     0         1         2         3         4         5   \
fitur                                                                       
jam           -0.000958  0.000422  0.009113  0.016088 -0.016612 -0.012709   
menit          0.008067 -0.012120 -0.006337  0.016783 -0.002817  0.000574   
indonesia     -0.010000  0.012687  0.005987  0.012903  0.012143  0.013448   
kilas          0.013274  0.011831 -0.006625 -0.015617  0.009710  0.011625   
news           0.014206 -0.008650  0.015042  0.009397 -0.011696  0.007068   
...                 ...       ...       ...       ...       ...       ...   
kotakabupaten  0.013327  0.002652  0.002008  0.003133 -0.013068 -0.015387   
libur         -0.005501 -0.004963  0.015005  0.005179  0.016734  0.009346   
hktb          -0.014825  0.012826  0.009712 -0.004105  0.017197 -0.010001   
jelajah       -0.012210 -0.003749 -0.000723 -0.010214 -0.012091 -0.014488   
aschina       -0.011090 -0.010163  0.012937  0.013762 -0.016622  0.002457   